In [4]:
# One-time installs
!apt install -y tesseract-ocr
!pip install -q pytesseract gtts pydub gradio transformers torchaudio

# Imports
import pytesseract
from PIL import Image
from gtts import gTTS
import os
from pydub import AudioSegment
from transformers import BlipProcessor, BlipForConditionalGeneration
import torch
import gradio as gr
import tempfile

# Captioning model
device = "cuda" if torch.cuda.is_available() else "cpu"
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base").to(device)

def describe_image(img):
    inputs = processor(images=img, return_tensors="pt").to(device)
    out = model.generate(**inputs)
    return processor.decode(out[0], skip_special_tokens=True)

def process_uploaded_images(images):
    clips = []
    log = []

    for idx, img_path in enumerate(images, start=1):
        try:
            img = Image.open(img_path.name).convert("RGB")
            text = pytesseract.image_to_string(img).strip()

            if text:
                narration = f"Let me speak about the image you uploaded.Wow I can read the text I see  : {text}."
            else:
                desc = describe_image(img)
                narration = f"Let me speak the image you uploaded. I cant see any text in the image but I can see: {desc}."

            tts = gTTS(text=narration, lang='en', tld='co.in')  # Indian accent
            tmp_mp3 = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
            tts.save(tmp_mp3.name)
            clips.append(AudioSegment.from_file(tmp_mp3.name))
            log.append(narration)

        except Exception as e:
            log.append(f"⚠️ Error processing image {idx}: {e}")

    # Merge all clips into one
    if clips:
        final_audio = clips[0]
        for segment in clips[1:]:
            final_audio += AudioSegment.silent(duration=700) + segment
        final_output_path = "/tmp/final_narration.mp3"
        final_audio.export(final_output_path, format="mp3")
        return final_output_path, "\n\n".join(log)
    else:
        return None, "No valid images processed."

# Gradio UI
ui = gr.Interface(
    fn=process_uploaded_images,
    inputs=gr.File(file_types=["image"], label="Upload Multiple Images", file_count="multiple"),
    outputs=[
        gr.Audio(label="🗣️ Final Narration (MP3)", type="filepath"),
        gr.Textbox(label="📝 Narration Log")
    ],
    title="🖼️ OCR + Image Caption to Speech",
    description="Upload multiple images. The system will extract text or describe the image, and speak everything in Indian female voice as one MP3 narration."
)

ui.launch()


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b3aea6b69f4b13218e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
